In [3]:
%%bash
pwd
# ensure that pip install pyspark==3.2.1 i.e. the SPARK version matches the pyspark version, otherwise feast apply fails with errorr "blah blah encryption" 

/home/cdsw/scripts


In [4]:
from datetime import datetime
import pandas as pd

from feast import FeatureStore
import os

# Note: see https://docs.feast.dev/getting-started/concepts/feature-retrieval for 
# more details on how to retrieve for all entities in the offline store instead
# feature_store = FeatureStore('/home/cdsw/homecredit_feature_repo')  # Initialize the feature store

entity_df = pd.DataFrame.from_dict(
    {
        # entity's join key -> entity values
        "sk_id_curr": [100002, 100003, 100004],
        # "event_timestamp" (reserved key) -> timestamps
        "event_timestamp": [
            datetime(2022, 12, 6, 12, 33, 20),
            datetime(2022, 12, 6, 12, 33, 20),
            datetime(2022, 12, 6, 12, 33, 20),
        ],
    }
)



store = FeatureStore(repo_path='/home/cdsw/homecredit_feature_repo')

training_df = store.get_historical_features(
    entity_df=entity_df,

    features=[
        "homecredit_appl_details_view:target",
        "homecredit_appl_details_view:flag_own_car",
        "homecredit_appl_details_view:flag_own_realty",
    ]
    # features=[
    #     "driver_hourly_stats:conv_rate",
    #     "driver_hourly_stats:acc_rate",
    #     "driver_hourly_stats:avg_daily_trips",
    # ],
).to_df()

feature_service = store.get_feature_service("homecredit_appl_details_Fservice")
training_df=store.get_historical_features(features=feature_service, entity_df = entity_df).to_df()
print("----- Example features -----\n")
print(training_df.head())

/home/cdsw/.local/lib/python3.7/site-packages/feast/infra/offline_stores/contrib/spark_offline_store/spark_source.py:78: RuntimeWarning: The spark data source API is an experimental feature in alpha development. This API is unstable and it could and most probably will be changed in the future.
  RuntimeWarning,
/home/cdsw/.local/lib/python3.7/site-packages/feast/infra/offline_stores/contrib/spark_offline_store/spark.py:125: RuntimeWarning: The spark offline store is an experimental feature in alpha development. Some functionality may still be unstable so functionality can change in the future.
  RuntimeWarning,
/home/cdsw/.local/lib/python3.7/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] =

----- Example features -----

   sk_id_curr     event_timestamp  index  target  code_gender  flag_own_car  \
0      100002 2022-12-06 12:33:20      0     1.0            0             0   
1      100003 2022-12-06 12:33:20      1     0.0            1             0   
2      100004 2022-12-06 12:33:20      2     0.0            0             1   

   flag_own_realty  cnt_children  amt_income_total  amt_credit  ...  \
0                0             0          202500.0    406597.5  ...   
1                1             0          270000.0   1293502.5  ...   
2                0             0           67500.0    135000.0  ...   

   cc_name_contract_status_sent_proposal_var  \
0                                        NaN   
1                                        NaN   
2                                        NaN   

   cc_name_contract_status_signed_max  cc_name_contract_status_signed_mean  \
0                                 NaN                                  NaN   
1                  

In [2]:
training_df.show()

AttributeError: 'SparkRetrievalJob' object has no attribute 'show'

In [2]:
# Offline Inferencing 

entity_df["event_timestamp"] = pd.to_datetime("now", utc=True)
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
).to_df()

print("\n----- Example features -----\n")
print(training_df.head())

/home/cdsw/.local/lib/python3.7/site-packages/feast/infra/offline_stores/contrib/spark_offline_store/spark_source.py:78: RuntimeWarning: The spark data source API is an experimental feature in alpha development. This API is unstable and it could and most probably will be changed in the future.
  RuntimeWarning,
/home/cdsw/.local/lib/python3.7/site-packages/feast/infra/offline_stores/contrib/spark_offline_store/spark.py:125: RuntimeWarning: The spark offline store is an experimental feature in alpha development. Some functionality may still be unstable so functionality can change in the future.
  RuntimeWarning,



----- Example features -----

   driver_id            event_timestamp  label_driver_reported_satisfaction  \
0       1001 2022-11-29 09:20:09.088035                                   1   
1       1002 2022-11-29 09:20:09.088035                                   5   
2       1003 2022-11-29 09:20:09.088035                                   3   

   conv_rate  acc_rate  avg_daily_trips  
0   0.193120  0.052746              802  
1   0.965542  0.295209              967  
2   0.551666  0.524331              388  
